### 1. Configuração e Carregamento

**Objetivo:** Importar as bibliotecas e carregar todas as tabelas das camadas Bronze e Silver que precisaremos.

In [0]:
from pyspark.sql.functions import col, to_date, substring, concat_ws, when, monotonically_increasing_id
from pyspark.sql.types import IntegerType

# --- CONFIGURAÇÃO ---
data_source = "sinasc"
root_path = "/lakehouse/health_insights_brasil"
bronze_table_name = "default.bronze_sinasc"
gold_path = f"{root_path}/gold/{data_source}"
fact_table_name = "fato_nascimento_sinasc"

# --- CARREGAR TABELAS ---
print("Carregando tabelas das camadas Bronze e Silver...")
df_bronze = spark.table(bronze_table_name)
df_dim_tempo = spark.table("default.dim_tempo_sinasc")
df_dim_parto = spark.table("default.dim_parto_sinasc")
df_dim_mae = spark.table("default.dim_mae_sinasc")
df_dim_localidade = spark.table("default.dim_localidade") 

print("✅ Todas as tabelas, incluindo dim_localidade, foram carregadas.")

Carregando tabelas das camadas Bronze e Silver...
✅ Todas as tabelas, incluindo dim_localidade, foram carregadas.


### 3. Adicionar Chaves Substitutas (Surrogate Keys) às Outras Dimensões
Nesta etapa, adicionamos uma chave numérica única, conhecida como Chave Substituta (Surrogate Key ou SK), para cada uma de nossas tabelas de dimensão. O objetivo principal é otimizar a performance das consultas. É muito mais rápido e eficiente para o sistema de banco de dados conectar a tabela de fatos às dimensões usando um único número (como sk_mae) do que usar uma combinação de múltiplas colunas de texto (como 'Casada' e '8 a 11 anos'). Além de acelerar as análises, esta prática também simplifica a estrutura da nossa tabela de fatos, mantendo-a mais limpa e organizada, contendo apenas as chaves e as métricas.

In [0]:
print("Adicionando chaves substitutas a todas as dimensões...")

df_dim_tempo = df_dim_tempo.withColumn("sk_tempo", monotonically_increasing_id())
df_dim_parto = df_dim_parto.withColumn("sk_parto", monotonically_increasing_id())
df_dim_mae = df_dim_mae.withColumn("sk_mae", monotonically_increasing_id())
df_dim_localidade = df_dim_localidade.withColumn("sk_localidade", monotonically_increasing_id())

print("✅ Chaves substitutas criadas.")

Adicionando chaves substitutas a todas as dimensões...
✅ Chaves substitutas criadas.


### 3. Preparar e Construir a Tabela Fato

**Objetivo:** Preparar a tabela bronze com as colunas decodificadas para o JOIN e construir a tabela fato.

In [0]:
print("Preparando e construindo a tabela fato...")
df_bronze_decoded = (df_bronze
    .withColumn("data_nascimento_corr", to_date(concat_ws("-", substring(col("DTNASC"), 5, 4), substring(col("DTNASC"), 3, 2), substring(col("DTNASC"), 1, 2))))
    .withColumn("sexo_desc", when(col("SEXO") == '1', "Masculino").when(col("SEXO") == '2', "Feminino").otherwise("Ignorado"))
    .withColumn("parto_desc", when(col("PARTO") == '1', "Vaginal").when(col("PARTO") == '2', "Cesáreo").otherwise("Ignorado"))
    .withColumn("anomalia_desc", when(col("IDANOMAL") == '1', "Sim").when(col("IDANOMAL") == '2', "Não").otherwise("Ignorado"))
    .withColumn("raca_cor_desc", when(col("RACACOR") == '1', "Branca").when(col("RACACOR") == '2', "Preta").when(col("RACACOR") == '3', "Amarela").when(col("RACACOR") == '4', "Parda").when(col("RACACOR") == '5', "Indígena").otherwise("Ignorado"))
    .withColumn("estado_civil_desc", when(col("ESTCIVMAE") == '1', "Solteira").when(col("ESTCIVMAE") == '2', "Casada").when(col("ESTCIVMAE") == '3', "Viúva").when(col("ESTCIVMAE") == '4', "Separado judicialmente/Divorciado").when(col("ESTCIVMAE") == '5', "União consensual").otherwise("Ignorado"))
    .withColumn("escolaridade_desc", when(col("ESCMAE") == '1', "Nenhuma").when(col("ESCMAE") == '2', "1 a 3 anos").when(col("ESCMAE") == '3', "4 a 7 anos").when(col("ESCMAE") == '4', "8 a 11 anos").when(col("ESCMAE") == '5', "12 anos e mais").otherwise("Ignorado"))
    .na.fill(value="Não Informado", subset=["sexo_desc", "parto_desc", "anomalia_desc", "raca_cor_desc", "estado_civil_desc", "escolaridade_desc"])
)
fato_nascimento = (df_bronze_decoded
    .join(df_dim_tempo, df_bronze_decoded.data_nascimento_corr == df_dim_tempo.data_completa, "left")
    .join(df_dim_parto, ["sexo_desc", "parto_desc", "anomalia_desc", "raca_cor_desc"], "left")
    .join(df_dim_mae, ["estado_civil_desc", "escolaridade_desc"], "left")
    .join(df_dim_localidade.alias("loc_nasc"), col("CODMUNNASC") == col("loc_nasc.id_municipio_ibge"), "left")
    .join(df_dim_localidade.alias("loc_res"), col("CODMUNRES") == col("loc_res.id_municipio_ibge"), "left")
    .select(
        col("NUMERODN").alias("id_nascimento"), col("sk_tempo"), col("sk_parto"), col("sk_mae"),
        col("loc_nasc.sk_localidade").alias("sk_localidade_nascimento"),
        col("loc_res.sk_localidade").alias("sk_localidade_residencia"),
        col("loc_nasc.nome_municipio").alias("nome_municipio_nascimento"),
        col("loc_res.nome_municipio").alias("nome_municipio_residencia"),
        col("PESO").cast(IntegerType()), col("APGAR1").cast(IntegerType()),
        col("APGAR5").cast(IntegerType()), col("IDADEMAE").cast(IntegerType()),
        col("CONSULTAS").cast(IntegerType()), col("data_nascimento_corr").alias("data_nascimento")
    )
)

Preparando e construindo a tabela fato...


### 4. Preparar e Construir a Tabela Fato com JOINs

**Objetivo:** Unir a tabela de nascimentos com todas as dimensões, incluindo a `dim_localidade`, para trazer as chaves substitutas (SKs) e os nomes dos municípios para a tabela fato.

In [0]:
print("Preparando e construindo a tabela fato...")

# Etapa 1: Prepara a tabela bronze com as colunas decodificadas para o JOIN
df_bronze_decoded = (df_bronze
    .withColumn("data_nascimento_corr", to_date(concat_ws("-", substring(col("DTNASC"), 5, 4), substring(col("DTNASC"), 3, 2), substring(col("DTNASC"), 1, 2))))
    .withColumn("sexo_desc", when(col("SEXO") == '1', "Masculino").when(col("SEXO") == '2', "Feminino").otherwise("Ignorado"))
    .withColumn("parto_desc", when(col("PARTO") == '1', "Vaginal").when(col("PARTO") == '2', "Cesáreo").otherwise("Ignorado"))
    .withColumn("anomalia_desc", when(col("IDANOMAL") == '1', "Sim").when(col("IDANOMAL") == '2', "Não").otherwise("Ignorado"))
    .withColumn("raca_cor_desc", when(col("RACACOR") == '1', "Branca").when(col("RACACOR") == '2', "Preta").when(col("RACACOR") == '3', "Amarela").when(col("RACACOR") == '4', "Parda").when(col("RACACOR") == '5', "Indígena").otherwise("Ignorado"))
    .withColumn("estado_civil_desc", when(col("ESTCIVMAE") == '1', "Solteira").when(col("ESTCIVMAE") == '2', "Casada").when(col("ESTCIVMAE") == '3', "Viúva").when(col("ESTCIVMAE") == '4', "Separado judicialmente/Divorciado").when(col("ESTCIVMAE") == '5', "União consensual").otherwise("Ignorado"))
    .withColumn("escolaridade_desc", when(col("ESCMAE") == '1', "Nenhuma").when(col("ESCMAE") == '2', "1 a 3 anos").when(col("ESCMAE") == '3', "4 a 7 anos").when(col("ESCMAE") == '4', "8 a 11 anos").when(col("ESCMAE") == '5', "12 anos e mais").otherwise("Ignorado"))
    .na.fill(value="Não Informado", subset=["sexo_desc", "parto_desc", "anomalia_desc", "raca_cor_desc", "estado_civil_desc", "escolaridade_desc"])
)

# Etapa 2: Une o DataFrame preparado com todas as dimensões
fato_nascimento = (df_bronze_decoded
    .join(df_dim_tempo, df_bronze_decoded.data_nascimento_corr == df_dim_tempo.data_completa, "left")
    .join(df_dim_parto, ["sexo_desc", "parto_desc", "anomalia_desc", "raca_cor_desc"], "left")
    .join(df_dim_mae, ["estado_civil_desc", "escolaridade_desc"], "left")
    .join(df_dim_localidade.alias("loc_nasc"), col("CODMUNNASC") == col("loc_nasc.id_municipio_ibge"), "left")
    .join(df_dim_localidade.alias("loc_res"), col("CODMUNRES") == col("loc_res.id_municipio_ibge"), "left")
    .select(
        col("NUMERODN").alias("id_nascimento"),
        col("sk_tempo"), 
        col("sk_parto"), 
        col("sk_mae"),
        col("loc_nasc.sk_localidade").alias("sk_localidade_nascimento"),
        col("loc_res.sk_localidade").alias("sk_localidade_residencia"),
        col("loc_nasc.nome_municipio").alias("nome_municipio_nascimento"),
        col("loc_res.nome_municipio").alias("nome_municipio_residencia"),
        col("PESO").cast(IntegerType()), 
        col("APGAR1").cast(IntegerType()),
        col("APGAR5").cast(IntegerType()), 
        col("IDADEMAE").cast(IntegerType()),
        col("CONSULTAS").cast(IntegerType()), 
        col("data_nascimento_corr").alias("data_nascimento")
    )
)

print("✅ Tabela fato construída com sucesso.")
display(fato_nascimento.limit(10))

Preparando e construindo a tabela fato...
✅ Tabela fato construída com sucesso.


id_nascimento,sk_tempo,sk_parto,sk_mae,sk_localidade_nascimento,sk_localidade_residencia,nome_municipio_nascimento,nome_municipio_residencia,PESO,APGAR1,APGAR5,IDADEMAE,CONSULTAS,data_nascimento
110001,161,36,6,null,32,null,Alta Floresta D'Oeste,3120,8,9,24,2,2024-02-14
110001,117,36,23,null,32,null,Alta Floresta D'Oeste,3564,8,9,29,4,2024-04-17
110001,26,36,20,null,32,null,Alta Floresta D'Oeste,2816,8,9,30,4,2024-05-29
110001,149,27,20,null,32,null,Alta Floresta D'Oeste,3126,8,9,14,4,2024-05-27
110001,72,36,3,null,32,null,Alta Floresta D'Oeste,3622,8,9,24,4,2024-05-13
110001,39,24,26,null,32,null,Alta Floresta D'Oeste,2935,8,9,29,4,2024-05-08
110001,73,44,19,null,32,null,Alta Floresta D'Oeste,3365,8,9,18,3,2024-05-01
110001,32,36,7,null,32,null,Alta Floresta D'Oeste,3298,8,9,29,4,2024-06-05
110002,165,18,20,null,6,null,Ariquemes,3240,8,9,20,4,2024-01-01
110002,165,36,21,null,681,null,Itapuã do Oeste,3960,9,9,40,4,2024-01-01


### 5. Salvar a Tabela Fato na Camada Gold

**Objetivo:** Persistir nossa tabela fato final na camada Gold, particionando por data para otimizar as consultas.

In [0]:

fact_path = f"{gold_path}/{fact_table_name}"
print(f"Limpando e salvando a tabela fato em: {fact_path}")
dbutils.fs.rm(fact_path, recurse=True)
spark.sql(f"DROP TABLE IF EXISTS default.{fact_table_name}")

(fato_nascimento.write
 .format("delta")
 .mode("overwrite")
 .partitionBy("data_nascimento") 
 .save(fact_path)
)

spark.sql(f"CREATE TABLE default.{fact_table_name} USING DELTA LOCATION '{fact_path}'")
print(f"✅ Tabela '{fact_table_name}' criada com sucesso na camada Gold.")

Limpando e salvando a tabela fato em: /lakehouse/health_insights_brasil/gold/sinasc/fato_nascimento_sinasc
✅ Tabela 'fato_nascimento_sinasc' criada com sucesso na camada Gold.
